In [ ]:
# 🚆 Simulador ferroviario avanzado con paradas y velocidades
!pip install ipyleaflet nest_asyncio --quiet

import nest_asyncio
nest_asyncio.apply()  # necesario en Jupyter/Colab

from ipyleaflet import Map, Marker, Polyline, CircleMarker
from ipywidgets import HTML
import pandas as pd
import numpy as np
import asyncio

# --- Estaciones ---
data = {
    'Nombre': ['PC','HY','ALL','GL','LS','RE','BF','ZA','TY','AD','BO','CHP','GW','GC','AK'],
    'Lat': [-34.62877725564810, -34.65357438966240, -34.66198550374880, -34.68575314566300, -34.70760762818290,
            -34.72694886248000, -34.74349080749730, -34.76093560701560, -34.77572506895700, -34.79776333881820,
            -34.82511425789840, -34.85937107390240, -34.88815394932920, -34.91791243047220, -34.98041453983010],
    'Lon': [-58.38054289577390, -58.37890608827360, -58.37662621882560, -58.38265921928540, -58.39076383735380,
            -58.39360056073690, -58.39541693417600, -58.39736114060200, -58.39658469300940, -58.39416220846470,
            -58.39139351273950, -58.38717748119720, -58.38313653354030, -58.38058564834750, -58.37381388778140]
}
df = pd.DataFrame(data)
coords = list(zip(df['Lat'], df['Lon']))

# --- Crear mapa ---
m = Map(center=[-34.8, -58.39], zoom=11)

# --- Dibujar estaciones ---
for _, row in df.iterrows():
    marker = CircleMarker(location=(row['Lat'], row['Lon']), radius=4, color='black',
                          fill_color='yellow', fill_opacity=1)
    m.add_layer(marker)

# --- Dibujar línea principal ---
linea = Polyline(locations=coords, color='gray', weight=3)
m.add_layer(linea)

# --- Función para interpolar coordenadas ---
def interpolar_ruta(ruta, pasos_por_tramo=10):
    smooth_coords = []
    for i in range(len(ruta)-1):
        lat_start, lon_start = ruta[i]
        lat_end, lon_end = ruta[i+1]
        for t in np.linspace(0,1,pasos_por_tramo, endpoint=False):
            lat = lat_start + t*(lat_end - lat_start)
            lon = lon_start + t*(lon_end - lon_start)
            smooth_coords.append((lat, lon))
    smooth_coords.append(ruta[-1])
    return smooth_coords

# --- Configurar trenes ---
trenes = [
    {'nombre': 'Tren 1', 'color': 'red', 'ruta': interpolar_ruta(coords), 'marker': None, 'pos':0, 'delay':0.1, 'pause':5},
    {'nombre': 'Tren 2', 'color': 'blue', 'ruta': interpolar_ruta(coords[::-1]), 'marker': None, 'pos':20, 'delay':0.08, 'pause':3},
    {'nombre': 'Tren 3', 'color': 'green', 'ruta': interpolar_ruta(coords), 'marker': None, 'pos':40, 'delay':0.12, 'pause':4},
    {'nombre': 'Tren 4', 'color': 'orange', 'ruta': interpolar_ruta(coords[::-1]), 'marker': None, 'pos':60, 'delay':0.09, 'pause':6},
]

# --- Crear markers ---
for tren in trenes:
    tren['marker'] = Marker(location=tren['ruta'][tren['pos']], draggable=False, title=tren['nombre'])
    m.add_layer(tren['marker'])

# --- Animación asincrónica con paradas ---
async def animar_trenes(trenes):
    while True:
        for tren in trenes:
            tren['pos'] = (tren['pos'] + 1) % len(tren['ruta'])
            tren['marker'].location = tren['ruta'][tren['pos']]

            # Verificar si estamos en una estación (aproximadamente)
            if any(np.isclose(tren['marker'].location[0], s[0], atol=1e-4) and
                   np.isclose(tren['marker'].location[1], s[1], atol=1e-4) for s in coords):
                await asyncio.sleep(tren['pause'])  # detenerse en estación

            await asyncio.sleep(tren['delay'])

# --- Ejecutar animación ---
loop = asyncio.get_event_loop()
loop.create_task(animar_trenes(trenes))

# --- Mostrar mapa ---
m


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00


Map(center=[-34.8, -58.39], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [ ]:
# 🚆 Simulador ferroviario con estado de trenes (compatible con Google Colab)
!pip install folium --quiet

import folium
from IPython.display import clear_output, display
import time
import pandas as pd
import numpy as np

# --- Estaciones ---
data = {
    'Nombre': ['PC','HY','ALL','GL','LS','RE','BF','ZA','TY','AD','BO','CHP','GW','GC','AK'],
    'Lat': [-34.62877725564810, -34.65357438966240, -34.66198550374880, -34.68575314566300, -34.70760762818290,
            -34.72694886248000, -34.74349080749730, -34.76093560701560, -34.77572506895700, -34.79776333881820,
            -34.82511425789840, -34.85937107390240, -34.88815394932920, -34.91791243047220, -34.98041453983010],
    'Lon': [-58.38054289577390, -58.37890608827360, -58.37662621882560, -58.38265921928540, -58.39076383735380,
            -58.39360056073690, -58.39541693417600, -58.39736114060200, -58.39658469300940, -58.39416220846470,
            -58.39139351273950, -58.38717748119720, -58.38313653354030, -58.38058564834750, -58.37381388778140]
}
df = pd.DataFrame(data)
coords = list(zip(df['Lat'], df['Lon']))

# --- Interpolación suave entre estaciones ---
def interpolar_ruta(ruta, pasos_por_tramo=8):
    smooth = []
    for i in range(len(ruta)-1):
        lat1, lon1 = ruta[i]
        lat2, lon2 = ruta[i+1]
        for t in np.linspace(0,1,pasos_por_tramo,endpoint=False):
            lat = lat1 + t*(lat2-lat1)
            lon = lon1 + t*(lon2-lon1)
            smooth.append((lat, lon))
    smooth.append(ruta[-1])
    return smooth

ruta_ida = interpolar_ruta(coords)
ruta_vuelta = interpolar_ruta(coords[::-1])

# --- Configurar trenes ---
trenes = [
    {"nombre": "Tren 1", "color": "red", "ruta": ruta_ida, "pos": 0, "velocidad": 1, "parada": 4, "estado": "En movimiento"},
    {"nombre": "Tren 2", "color": "blue", "ruta": ruta_vuelta, "pos": 30, "velocidad": 1, "parada": 3, "estado": "En movimiento"},
    {"nombre": "Tren 3", "color": "green", "ruta": ruta_ida, "pos": 60, "velocidad": 1, "parada": 5, "estado": "En movimiento"},
]

# --- Simulación ---
def mover_trenes(trenes, pasos=200, delay=0.4):
    for i in range(pasos):
        mapa = folium.Map(location=[-34.8, -58.39], zoom_start=11)

        # Dibujar línea y estaciones
        folium.PolyLine(coords, color="gray", weight=3).add_to(mapa)
        for _, row in df.iterrows():
            folium.CircleMarker([row['Lat'], row['Lon']], radius=4,
                                color="black", fill=True, fill_color="yellow").add_to(mapa)

        # Actualizar trenes
        for tren in trenes:
            # Si está detenido, cuenta los ciclos de parada
            if tren['velocidad'] == 0:
                tren['parada'] -= 1
                if tren['parada'] <= 0:
                    tren['velocidad'] = 1
                    tren['estado'] = "En movimiento"
                    tren['parada'] = np.random.randint(3,6)  # próxima parada diferente
            else:
                tren['pos'] = (tren['pos'] + tren['velocidad']) % len(tren['ruta'])
                pos = tren['ruta'][tren['pos']]

                # Verificar si está en estación
                for est in coords:
                    if abs(pos[0]-est[0]) < 0.0003 and abs(pos[1]-est[1]) < 0.0003:
                        tren['velocidad'] = 0
                        tren['estado'] = "En estación"
                        break

            pos = tren['ruta'][tren['pos']]
            folium.CircleMarker(pos, radius=6, color=tren['color'],
                                fill=True, fill_color=tren['color'],
                                popup=f"{tren['nombre']} ({tren['estado']})").add_to(mapa)

        clear_output(wait=True)
        display(mapa)
        time.sleep(delay)

# --- Ejecutar simulación ---
mover_trenes(trenes, pasos=150, delay=0.5)
